In [5]:
import nltk                          #extractive summary
import math
from operator import eq 

def OpinosisGraph(Z, G, PRI):  
    for i in range(len(Z)):  
        for j in range(len(Z[i])):  
            LABEL = Z[i][j]
            PID = j
            SID = i
            if LABEL in G:
                PRI[LABEL].append((SID, PID))
            else:
                G[LABEL] = list()
                PRI[LABEL] = [(SID, PID)]
            if j > 0 and LABEL not in G[Z[i][j-1]]:
                G[Z[i][j-1]].append(Z[i][j])
                
def VSN(node):   
    global PRI1
    sigma_vsn = 15
    ele = PRI1[node]
    sum = 0
    for i in range(len(ele)):
        sum = sum + ele[i][1]
    return (sum/len(ele)) <= sigma_vsn

def collapsible(node):  
    if(node[1] == 'VERB'):
        return True
    else: 
        return False

def valid_sent(sent):     
    return True

def check_inter(PRIa,PRIb):  
    PRIc = []
    sigma_gap = 4
    for first in PRIa:
        for second in PRIb:
            if(first[0] == second[0] and abs(first[1]-second[1]) <= sigma_gap):
                PRIc.append(second)
    return PRIc

def eliminate_dup(clist): 
    buf = []
    for ele in clist:
        if ele[0] not in buf:
            buf.append(ele[0])
        else:
            clist.remove(ele)
    return clist

def common_elements(list1, list2):  
    count = 0
    for element in list1:
        if element in list2:
            count= count+1
    return count
                                            
def jaccard_fail(a,b):   
    sigma_j = 0.3
    inter = common_elements(a,b)
    return ((inter/(len(a)+len(b)-inter)) >= sigma_j)
    

def eliminate_dup(clist): 
    buf = []
    first = True
    for ele in clist:
        temp = []
        similar = False
        if(first):
            buf.append(ele)
            first = False
        else:
            for seen in buf:
                if jaccard_fail(seen[0],ele[0]):
                    similar = True
                    if ele[1]>seen[1]:                           
                        temp.append(seen)
            if len(temp) != 0:  
                for seen in temp:
                    buf.remove(seen)
                buf.append(ele)
            elif similar == False:
                buf.append(ele)  
    return buf



def find_avg_score(clist):    
    if len(clist)==0:
        return 0
    sum = 0
    for ele in clist:
        sum = sum + ele[1]
    return sum/len(clist)

def find_sum(C):            
    winner = (None,-1)
    runner_up = (None,-1)
    for ele in C:
        if ele[1] > winner[1]:
            if winner[1] > runner_up[1]:
                runner_up = winner
            winner = ele
        elif ele[1] > runner_up[1]:
            runner_up = ele
    ans1 = ""
    for words in winner[0]:
        ans1 = ans1 + words[0] + " "
    ans1 = ans1[0].upper() + ans1[1:(len(ans1)-1)]
    ans2 = ""
    for words in runner_up[0]:
        ans2 = ans2 + words[0] + " "
    ans2 = ans2[0].upper() + ans2[1:(len(ans2)-1)]
    return ans1,ans2

def Traverse(clist,vk,score,PRI_overlap,sent,length):    
    #print(sent)
    global G1
    global PRI1
    global sentence_number
    global neighbor
    sigma_r = 0
    redundancy = len(PRI_overlap)
    if length>100:        
        return
    if (redundancy / length >= sigma_r):  
        if (neighbor == len(original_sentence[sentence_number])):
            if (valid_sent(sent)):
                finalscore = score/length
                clist.append((sent,finalscore))
                return 
        else:
            vn = original_sentence[sentence_number][neighbor]
            neighbor += 1
            PRI_new = check_inter(PRI_overlap,PRI1[vn])
            redundancy = len(PRI_new)
            new_sent = sent+[vn]
            L = length + 1
            new_score = score + redundancy  
            Traverse(clist,vn,new_score,PRI_new,new_sent,L)
    
def first_lower(s):     
    return s.lower()
        
def read_text(file_name):    
    redundant = '<br />'
    review = []
    with open(file_name,'r') as file:
        for article in file:
            article = article.replace(redundant," ")
            sentences = nltk.sent_tokenize(article) 
            for sentence in sentences:
                sentence = first_lower(sentence)
                review.append(nltk.pos_tag(nltk.word_tokenize(sentence),tagset='universal'))           
    file.close()
    return review


In [6]:
import nltk
import os
filenames = [f for f in os.listdir('.') if os.path.isfile(f)]
filenames.remove('negative-words.txt')
filenames.remove('positive-words.txt')
for name in filenames:
    a,b = name.split('.')
    original_sentence = []
    if b == 'txt':
        print(name)
        Z1 = read_text(name)
        Z2 = []
        for i in Z1:
            original_sentence.append(i)
            Z2.append(i[0])
        G1 = dict()
        PRI1 = dict()
        OpinosisGraph(Z1, G1, PRI1)   #paper Alg2
        candidates = []
        for i in range(len(Z2)):
            keys = Z2[i]
            sentence_number = i
            neighbor = 1
            if(VSN(keys)):
                clist = []
                sum_sent = [keys]
                score = 0
                Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                candidates = candidates + clist
        C = eliminate_dup(candidates)
        summary = find_sum(C)
        print("summary1: ",summary[0])
        print("summary2: ",summary[1])

00_3.txt
summary1:  Starts out with a opening scene that is a terrific example of absurd comedy .
summary2:  A formal orchestra audience is turned into an insane , violent mob by the crazy chantings of it 's singers .
00_9.txt
summary1:  A classic line : inspector : i 'm here to sack one of your teachers .
summary2:  The scramble to survive financially , the insightful students who can see right through their pathetic teachers ' pomp , the pettiness of the whole situation , all remind me of the schools i knew and their students .
01_1.txt
summary1:  It also plugs in every cliche in the book .
summary2:  This movie is so wasteful of talent , it is truly disgusting .
01_7.txt
summary1:  There 's the idiotic principal , mr. bip , the nervous maths teacher and many others .
summary2:  The cast is also fantastic , lenny henry 's gina yashere , eastenders chrissie watts , tracy-ann oberman , smack the pony 's doon mackichan , dead ringers ' mark perry and blunder 's nina conti .
02_1.txt
sum

summary1:  Overall , i ca n't recommend this movie , not for buying , renting , or even seeing for free .
summary2:  Also counting against this film are some constant camera tricks that generally seemed annoying and out-of-place , such as slow-motion , fast-motion , freeze-frames and echos .
16_9.txt
summary1:  I would have preferred a bit of true whodunit mystery in this , but hey , when a short is this good , who 's going to complain .
summary2:  Of the elements that make this the best at this point , i have to say # 1 is christine mcintire .
17_3.txt
summary1:  Nine minutes of psychedelic , pulsating , often symmetric abstract images , are enough to drive anyone crazy .
summary2:  I did spot a full-frame eye at the start , and later some birds silhouetted against other colors .
17_7.txt
summary1:  Directed by emilio p. miraglia who , one year earlier , also made the very interesting `` la notte che evelyn usci della tomba '' ( see also my comment on that one ) , the film starts off 